## WFP-01-02-01 Sentinel-2 reflectances and vegetation indices

### <a name="quicklink">Quick link

* [Objective](#objective)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [License](#license)

### <a name="objective">Objective 

As a land cover change analyst I want to derive the vegetation indices from Sentinel-2 products.

Co-located timeseries of:

* Sentinel-2 Level 2A reflectances
* Indices NDVI, NDWI, MNDWI, NDBI

### <a name="data">Data 

SENTINEL data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. Radar data will be delivered within an hour of reception for Near Real-Time (NRT) emergency response, within three hours for NRT priority areas and within 24 hours for systematically archived data.

The data used are Sentinel-2 Level-1C products: Top of atmosphere reflectances in fixed cartographic geometry (combined UTM projection and WGS84 ellipsoid). Level-1C images are a set of tiles of 100 sq km, each of which is approximately 500 MB. These products contain applied radiometric and geometric corrections (including orthorectification and spatial registration). 

The spectral bands of Sentinel-2 Level-1C products are:

| S-2 band                | 1   | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 8a  | 9   | 10   | 11   | 12   |
|-------------------------|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|------|------|------|
| Central wavelength (nm) | 443 | 490 | 560 | 665 | 705 | 740 | 783 | 842 | 865 | 945 | 1375 | 1610 | 2190 |
| Bandwidth (nm)          | 20  | 65  | 35  | 30  | 15  | 15  | 20  | 115 | 20  | 20  | 30   | 90   | 180  |
| Spatial resolution (m)  | 60  | 10  | 10  | 10  | 20  | 20  | 20  | 10  | 20  | 60  | 60   | 20   | 20   |

### <a name="service">Service Definition

In [2]:
service = dict([('title', 'wfp-01-02-01'),
                ('abstract', 'WFP-01-02-01 Sentinel-2 reflectances and vegetation indices'),
                ('id', 'ewf-wfp-01-02-01')])

### <a name="parameter">Parameter Definition 

**Resolution**

Target resolution expressed in meters: 10, 20 or 60

In [3]:
resolution = dict([('id', 'resolution'),
                   ('value', '10'),
                   ('title', 'Target resolution'),
                   ('abstract', 'Target resolution (10m, 20m or 60m)')])

**Area of interest WKT**

Area of interest expressed in Well-Known Text

In [4]:
wkt = dict([('id', 'wkt'),
            ('value', 'POLYGON((14.9997721265064 38.0212298555682,15.0339421243074 38.1469295398089,15.076722011609 38.2943830685215,15.1196939590664 38.4418167678655,15.1628416993738 38.5892613166416,15.2061510925436 38.7366798078584,15.2388261265461 38.8477042183055,16.2649601148006 38.8421486728812,16.2478654087227 37.8528295032852,14.9997726359092 37.8594419593273,14.9997721265064 38.0212298555682))'),
            ('title', 'Protected Area wkt'),
            ('abstract', 'Protected Area wkt')])


** Sentinel-2 flag expression**

Flag expression for pixel selection

In [5]:
flag_expr = dict([('id', 'flag_expr'),
                  ('value', '( saturated_l1a_B4 or scl_water )'),
                  ('title', 'Flag expression for pixel exclusion'),
                  ('abstract', 'Flag expression for pixel exclusion (e.g. saturated_l1a_B4 will exclude pixels having the flag saturated_l1a_B4 set)')])

### <a name="runtime">Runtime parameter definition

**Input identifier**

This is the Sentinel-2 product identifier

In [6]:
input_identifier = 'S2B_MSIL2A_20180803T094029_N0208_R036_T33SWC_20180803T141742'

**Input reference**

This is the Sentinel-2 catalogue reference

In [7]:
input_reference = 'https://catalog.terradue.com/sentinel2/search?uid=S2B_MSIL2A_20180803T094029_N0208_R036_T33SWC_20180803T141742'

**Data path**

This path defines where the data is staged-in. 

In [11]:
data_path = '/data2'

### <a name="workflow">Workflow

#### Import the packages required for processing the Sentinel-1 backscatter

In [12]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import os
import sys

import dateutil.parser as parser

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import snappy

import gc

import gdalnumeric

import gdal
import osr
import numpy as np

#### Open the Sentinel-2 product

In [13]:
s2prd = "%s/%s/%s.SAFE/MTD_MSIL2A.xml" % (data_path, input_identifier, input_identifier)
product = snappy.ProductIO.readProduct(s2prd)

width = product.getSceneRasterWidth()
height = product.getSceneRasterHeight()
name = product.getName()
description = product.getDescription()
band_names = product.getBandNames()



In [14]:
product_date = parser.parse(product.getStartTime().toString()).date()
output_date = '%s%02d%02d' % (product_date.year, product_date.month, product_date.day)


#### Resample 

In [15]:
if resolution['value'] == '10':
    reference_band = 'B4'

if resolution['value'] == '20':
    reference_band = 'B5'
    
if resolution['value'] == '60':
    reference_band = 'B1' 

In [16]:
snappy.GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

HashMap = snappy.jpy.get_type('java.util.HashMap')
   
parameters = HashMap()
parameters.put('referenceBand', reference_band)

resample = snappy.GPF.createProduct('Resample', parameters, product)

In [39]:
# NDVI, NDWI, MNDWI, NDBI

# TODO check expressions
ndvi_expr = '((B8 - B4) / (B8 + B4)) * 10000'
ndwi_expr = '((B8A - B11) / ( B8A + B11)) * 10000'
mndwi_expr = '((B11 - B4) / (B11 + B4 )) * 10000'
ndbi_expr = '((B11 - B4) / (B11 + B4 )) * 10000'

if flag_expr['value']:

    ndvi_expr = '! %s ? %s : -20000' % (flag_expr['value'], ndvi_expr)
    ndwi_expr = '! %s ? %s : -20000' % (flag_expr['value'], ndwi_expr)
    mndwi_expr = '! %s ? %s : -20000' % (flag_expr['value'], mndwi_expr)
    ndbi_expr = '! %s ? %s : -20000' % (flag_expr['value'], ndbi_expr) 

In [40]:
band_names = resample.getBandNames()
list(band_names)[0:12]

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']

In [41]:
target_bands = list(band_names)[0:12] + ['ndvi', 
                                         'ndwi',
                                         'mndwi',
                                         'ndbi',
                                         'quality_cloud_confidence',
                                         'quality_snow_confidence',
                                         'quality_scene_classification']

In [42]:
BandDescriptor = snappy.jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')

targetBands = snappy.jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(target_bands))

for index, band in enumerate(target_bands):
     
    targetBand = BandDescriptor()
    
    if band == 'ndvi':
       
        targetBand.expression = ndvi_expr 
    
    elif band == 'ndwi':
        
        targetBand.expression = ndwi_expr 
    
    elif band == 'mndwi':
        
        targetBand.expression = mndwi_expr 

    elif band == 'ndbi':
    
         targetBand.expression = ndbi_expr 
    
    else:
    
        targetBand.expression = band

    targetBand.name = band
    targetBand.type = 'float32'
    
    targetBands[index]= targetBand
        

parameters = HashMap()
parameters.put('targetBands', targetBands)

result = snappy.GPF.createProduct('BandMaths', parameters, resample)

#### Subset

In [ ]:
WKTReader = snappy.jpy.get_type('com.vividsolutions.jts.io.WKTReader')

geom = WKTReader().read(wkt['value'])

parameters = HashMap()
parameters.put('copyMetadata', True)
parameters.put('geoRegion', geom)
    
subset_result = snappy.GPF.createProduct('Subset', parameters, result)


output_name = '%s_VI_BOA' % input_identifier

snappy.ProductIO.writeProduct(subset_result, output_name, 'GeoTIFF-BigTIFF')

### <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.